# P-Median

Explicar aqui un poco el contexto de p-median y quizas pegar su formula

### 1. Libraries

In [1]:
import numpy as np

### 2. Configure project

Explicar como se configura


In [2]:
config = {
    "distance_data_path": "data/test/distance.txt",
    "demand_data_path": "data/test/demand.txt",
    "dimensions": 6,
    "type_initial_solution": "random"
}

### 3. Open data file

In [3]:
def readORLibDataFile(data_path = ""):
    if data_path == "":
        print("Empty data path, change it in config step.")
        return []
    
    data = []
    raw_data = open(data_path, "r")
    for line in raw_data:
        line = line.strip().split(" ")
        line = list(map(int, line))
        data.append(line)
        
    return data

def readTestDataFile(data_path = "", dim = 0):
    if data_path == "":
        print("Empty data path, change it in config step.")
        return []
    
    if dim <= 0:
        print("Dimensions are wrong.")
        return []
    
    data = []
    
    raw_data = open(data_path, "r")
    for line in raw_data:
        line = line.strip().split(",")
        line = list(map(int, line))
        data.append(line)
        
    return data

def print_matrix(m):
    l = len(m)
    for i in range(l):
        s = ""
        for j in range(l):
            s += str(m[i][j]) + " "
        print(s)

In [4]:
distance = readTestDataFile(config["distance_data_path"], config["dimensions"])
#demand = readTestDataFile(config["demand_data_path"], config["dimensions"])

### 4. Initial Solutions

In [5]:
def randomInitialSolution(p, size):
    x = np.zeros(size, dtype=int)
    x[:p] = 1
    np.random.shuffle(x)
    return x

### 4. P-Median Objective Function

In [34]:
def selectFacility(facility, n, solution, distance):
    #Determinados la mayor distancia de la fila
    min_value = max(distance[facility])
    min_index = 0
    
    for i in range(n):
        if solution[i] == 1:
            if distance[facility][i] < min_value:
                min_value = distance[facility][i]
                min_index = i
    
    return min_index
    
    
    

def pMedianObjectiveFunction(permutation, distance, demand):
    n = len(distance[0])
    selected_facilities = []
    
    total = 0
    #Loop over locations
    for i in range(n):
        #Indice de donde se alimenta la planta
        min_index = selectFacility(i, n, permutation, distance)
        
        #Creamos una fila y guardamos la distancia
        aux = [0] * n
        aux[min_index] = 1
        selected_facilities.append(aux)
        
        #Obtenemos el valor de la demanda * la distancia minima, lo agregamos al total
        total += demand[i] * distance[i][min_index]
        
               
    #print(total)
    return total, selected_facilities

In [69]:
def area_facility(n, index, selected_facilities, distance, demand):
    area = []
    
    temp = [j[index] for j in selected_facilities]
    for j in range(n):
        if temp[j] == 1:
            area.append(temp)
        else:
            area.append([0] * n)
        
    return area


def calculate_cost(area, distance, demand):
    n = len(area[0])
    temp = [0] * n
    for i in range(n):
        for j in range(n):
            temp[j] += area[i][j] * distance[i][j] * demand[i]
        
    
    return temp

def changeSelectedFacility(index, selected_facilities, new_cost):
    temp = [j[index] for j in selected_facilities]
    m = len(temp)
    
    lowest_cost_value = max(new_cost)
    lowest_cost_index = -1
    
    for i in range(m):
        if new_cost[i] != 0 and new_cost[i] < lowest_cost_value:
            lowest_cost_value = new_cost[i]
            lowest_cost_index = i
    
    return lowest_cost_value, lowest_cost_index

### 5. Heurística de Maranzana

In [76]:
def createInitialSolution(t, p, size):
    if t == "random":
        return randomInitialSolution(p, size)
    else:
        print("not implemented yet")
        return []



def solveMaranzana(type_initial_solution, p, distance, demand):
    #Create initial solution
    
    #solution = createInitialSolution(type_initial_solution, p, len(demand))
    #Borrar, esto es por mientras
    solution = type_initial_solution
    
    #Calculate of_value of initial solution
    of_value, selected_facilities = pMedianObjectiveFunction(solution, distance, demand)
    
    #Chequear mejor solución para cada lugar seleccionado
    m = len(solution)
    is_changed = True

    while is_changed is True:
        solution_changed = False
        new_solution = solution.copy()
        for i in range(m):
            if solution[i] == 1:
                area = area_facility(m, i, selected_facilities, distance, demand)
                new_cost = calculate_cost(area, distance, demand)
                lowest_cost_value, lowest_cost_index = changeSelectedFacility(i, selected_facilities, new_cost)
                
                if i != lowest_cost_index and lowest_cost_index != -1:
                    new_solution[i] = 0
                    new_solution[lowest_cost_index] = 1
                    solution_changed = True
                
        if solution_changed:
            of_value, selected_facilities = pMedianObjectiveFunction(new_solution, distance, demand)
            solution = new_solution
        else:
            is_changed = False
    
    return solution, of_value, selected_facilities

In [82]:
solution = [0,0,1,1,0,0]
demand = [7,5,3,4,5,10]            

sol, of_value, sf = solveMaranzana(solution, 2, distance, demand)
print(of_value)
print(sol)
print_matrix(sf)

157
[0, 1, 0, 0, 0, 1]
0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 0 1 
0 0 0 0 0 1 
0 0 0 0 0 1 
